# Bayes Estimator

### <font color='orange'>Universidad Autónoma de Yucatán</font> _Facultad de Matemáticas_

**Teacher:** Dr. Arturo Espinosa Romero <[eromero@correo.uady.mx](mailto:eromero@correo.uady.mx)>

**Student:** Ing. Dayan Bravo Fraga <[dayan3847@gmail.com](mailto:dayan3847@gmail.com)>

# Practice 5: Implicit Extended Kalman Filter for Ball Tracking

## GitHub: [Practice 5](https://github.com/dayan3847/bayes_estimator/tree/main/practice5-ball_tracking)

In [3]:
import sympy as sp

In [4]:
#@title Estado de sistema
X, Y, Z, = sp.symbols('X Y Z')
dX, dY, dZ = sp.symbols('\dot{X} \dot{Y} \dot{Z}')
# El estado esta compuesto por las coordenadas reales y las velocidades
XX = sp.Matrix([
    [X],
    [Y],
    [Z],
    [dX],
    [dY],
    [dZ],
])

sp.Eq(sp.Symbol('\mathbb{X}'), XX, evaluate=False)

Eq(\mathbb{X}, Matrix([
[      X],
[      Y],
[      Z],
[\dot{X}],
[\dot{Y}],
[\dot{Z}]]))

In [4]:
#@title Matriz A "transitionMatrix"
dt = sp.Symbol('\Delta t')
# Esta seria la matrix de transformacion de estado
AA = sp.Matrix([
    [1, 0, 0, dt, 0, 0],
    [0, 1, 0, 0, dt, 0],
    [0, 0, 1, 0, 0, dt],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1],
])
sp.Eq(sp.Symbol('\mathbb{A}'), AA, evaluate=False)

Eq(\mathbb{A}, Matrix([
[1, 0, 0, \Delta t,        0,        0],
[0, 1, 0,        0, \Delta t,        0],
[0, 0, 1,        0,        0, \Delta t],
[0, 0, 0,        1,        0,        0],
[0, 0, 0,        0,        1,        0],
[0, 0, 0,        0,        0,        1]]))

In [5]:
#@title "Z" Medicion
x, y, r = sp.symbols('x y r')
dx, dy = sp.symbols('\dot{x} \dot{y}')
# La medicion esta compuesta por las coordenadas de la camara y el tamanno del radio (en metros)
ZZ = sp.Matrix([
    [x],
    [y],
    [r],
    [dx],
    [dy],
])
sp.Eq(sp.Symbol('\mathbb{Z}'), ZZ, evaluate=False)

Eq(\mathbb{Z}, Matrix([
[      x],
[      y],
[      r],
[\dot{x}],
[\dot{y}]]))

In [13]:
# @title Funcion h
Rm = sp.Symbol('Rm')  # Radio real de la pelota en metros
h_xz = sp.Matrix([
    [x - X / Z],
    [y - Y / Z],
    [r - Rm / Z],
    [(dx - (-X * dZ / Z ** 2 + dX / Z)).subs(X / Z, x)],
    [(dy - (-Y * dZ / Z ** 2 + dY / Z)).subs(Y / Z, y)],
])
sp.Eq(sp.Eq(sp.Function('h')(sp.Symbol('\mathbb{X,Y}'), evaluate=False), h_xz, evaluate=False),
      sp.Symbol('\emptyset'), evaluate=False)

Eq(Eq(h(\mathbb{X,Y}), Matrix([
[                         -X/Z + x],
[                         -Y/Z + y],
[                        -Rm/Z + r],
[\dot{x} - \dot{X}/Z + \dot{Z}*x/Z],
[\dot{y} - \dot{Y}/Z + \dot{Z}*y/Z]])), \emptyset)

In [14]:
# @title Jacobiano de h(X,Z) con respecto a X
HH = h_xz.jacobian(XX)
sp.Eq(sp.Symbol(r'\mathbb{H}'), HH, evaluate=False)

Eq(\mathbb{H}, Matrix([
[-1/Z,    0,                        X/Z**2,    0,    0,   0],
[   0, -1/Z,                        Y/Z**2,    0,    0,   0],
[   0,    0,                       Rm/Z**2,    0,    0,   0],
[   0,    0, \dot{X}/Z**2 - \dot{Z}*x/Z**2, -1/Z,    0, x/Z],
[   0,    0, \dot{Y}/Z**2 - \dot{Z}*y/Z**2,    0, -1/Z, y/Z]]))

In [15]:
# @title Jacobiano de h(X,Z) con respecto a Z
JJ = h_xz.jacobian(ZZ)
sp.Eq(sp.Symbol(r'\mathbb{J}'), JJ, evaluate=False)

Eq(\mathbb{J}, Matrix([
[        1,         0, 0, 0, 0],
[        0,         1, 0, 0, 0],
[        0,         0, 1, 0, 0],
[\dot{Z}/Z,         0, 0, 1, 0],
[        0, \dot{Z}/Z, 0, 0, 1]]))

In [19]:
rows, cols = h_xz.shape
for r in range(rows):
    line = ''
    for c in range(cols):
        line += str(
            h_xz[r, c]
            .subs(dX, sp.Symbol('dX'))
            .subs(dY, sp.Symbol('dY'))
            .subs(dZ, sp.Symbol('dZ'))
            .subs(dx, sp.Symbol('dx'))
            .subs(dy, sp.Symbol('dy'))
        ) + ', '
    print(line)

-X/Z + x, 
-Y/Z + y, 
-Rm/Z + r, 
dx - dX/Z + dZ*x/Z, 
dy - dY/Z + dZ*y/Z, 


In [16]:
rows, cols = HH.shape
for r in range(rows):
    line = ''
    for c in range(cols):
        line += str(
            HH[r, c]
            .subs(dX, sp.Symbol('dX'))
            .subs(dY, sp.Symbol('dY'))
            .subs(dZ, sp.Symbol('dZ'))
        ) + ', '
    print(line)

-1/Z, 0, X/Z**2, 0, 0, 0, 
0, -1/Z, Y/Z**2, 0, 0, 0, 
0, 0, Rm/Z**2, 0, 0, 0, 
0, 0, dX/Z**2 - dZ*x/Z**2, -1/Z, 0, x/Z, 
0, 0, dY/Z**2 - dZ*y/Z**2, 0, -1/Z, y/Z, 


In [17]:
rows, cols = JJ.shape
for r in range(rows):
    line = ''
    for c in range(cols):
        line += str(
            JJ[r, c]
            .subs(dZ, sp.Symbol('dZ'))
        ) + ', '
    print(line)

1, 0, 0, 0, 0, 
0, 1, 0, 0, 0, 
0, 0, 1, 0, 0, 
dZ/Z, 0, 0, 1, 0, 
0, dZ/Z, 0, 0, 1, 
